    TOPIC: Beyond News Headlines and TF-IDF: Enhancing Text-Based Forecasting Models with Validated Collocations and Improved Attention.
    Author : Gabriel Appau Abeyie.

This code is dedicated to determining the apporpriate parameter values for the tfidf.

In [1]:
# Libraries for parsing data
import os
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from lxml import etree
import sys
import glob
import nltk
import unicodedata
import re
import pandas as pd
import numpy as np
from math import log2
import pickle as pk
import matplotlib.pyplot as plt
import warnings
import datetime
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
import numpy as np

In [20]:
def get_ngram_df(df, ngram_range, min_df=0.0, tfidf_min=0.0, split_ratio=0.625):
    # Split the data based on split ratio
    split_index = int(len(df) * split_ratio)
    df_train = df.iloc[:split_index]
    df_test = df.iloc[split_index:]

    # Create and fit the TfidfVectorizer
    vectorizer_tfidf = TfidfVectorizer(analyzer="word", ngram_range=ngram_range, min_df=min_df, stop_words='english')
    X_tfidf_train = vectorizer_tfidf.fit_transform(df_train['cleaned_Text'])

    # Calculate total TF-IDF for each n-gram and filter by tfidf_min
    columns = vectorizer_tfidf.get_feature_names_out()
    df_tfidf_train = pd.DataFrame(X_tfidf_train.toarray(), columns=columns)
    df_total_tfidf = df_tfidf_train.sum().reset_index()
    df_total_tfidf.columns = ['ngram', 'total_tfidf']
    filtered_ngrams = df_total_tfidf[df_total_tfidf['total_tfidf'] >= tfidf_min]['ngram'].tolist()

    # Create and fit the CountVectorizer
    vectorizer_count = CountVectorizer(vocabulary=filtered_ngrams, ngram_range=ngram_range, stop_words='english')
    X_count_train = vectorizer_count.transform(df_train['cleaned_Text'])
    X_count_test = vectorizer_count.transform(df_test['cleaned_Text'])

    # Convert to dataframes
    columns = vectorizer_count.get_feature_names_out()
    df_count_train = pd.DataFrame(X_count_train.toarray(), columns=columns, index=df_train.index)
    df_count_test = pd.DataFrame(X_count_test.toarray(), columns=columns, index=df_test.index)

    df_count_train['Target'] = df['Target'].loc[df_count_train.index]
    df_count_test['Target'] = df['Target'].loc[df_count_test.index]

    return df_count_train, df_count_test

In [21]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

def evaluate_model(df_train, df_test, target_column):
    # Separate features and target
    X_train = df_train.drop(columns=[target_column])
    y_train = df_train[target_column]
    X_test = df_test.drop(columns=[target_column])
    y_test = df_test[target_column]

    # Standardize the features
    scalar = StandardScaler()
    scalar.fit(X_train)
    X_train = scalar.fit_transform(X_train)
    X_test = scalar.transform(X_test)

    # Define and train the model
    model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=tscv, max_iter=1000, tol=0.001)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Evaluate predictions using your chosen metric (e.g. MSE)
    score = -mean_squared_error(y_test, predictions)  # We use a negative sign as grid search considers larger scores as better

    return score

def grid_search(df, ngram_range, target_column, split_ratio=0.625):
    # Define the parameter grid
    min_df_values = np.arange(0.01, 0.21, 0.01)  # values from 0.01 to 0.2, step size 0.01
    tfidf_min_values = np.arange(0.5, 2.1, 0.1)  # values from 0.5 to 2.0, step size 0.1

    best_score = -np.inf
    best_params = None

    # Iterate over all combinations of min_df and tfidf_min
    for min_df, tfidf_min in product(min_df_values, tfidf_min_values):
        # Get train and test dataframes
        df_count_train, df_count_test = get_ngram_df(df, ngram_range, min_df, tfidf_min, split_ratio)

        # Evaluate the model on the validation set
        score = evaluate_model(df_count_train, df_count_test, target_column)

        # Update best score and best parameters
        if score > best_score:
            best_score = score
            best_params = (min_df, tfidf_min)

    print(f'Best score: {best_score}')
    print(f'Best parameters: min_df={best_params[0]}, tfidf_min={best_params[1]}')

In [11]:
Nymex = pd.read_csv('E:/Python/Untitled Folder/DCOILWTICO.csv')
Target = Nymex[['Target']]

In [22]:
df = pd.concat([df1, Target['Target']], axis=1)
grid_search(df, ngram_range = (1,2), target_column='Target')

Best score: -0.0010165487015585766
Best parameters: min_df=0.060000000000000005, tfidf_min=0.7999999999999999


In [23]:
NewsHeadlines = pd.read_csv('E:/Python/Untitled Folder/NewHeadlines.csv')
NewsHeadlines = pd.concat([NewsHeadlines, Target['Target']], axis=1)
grid_search( NewsHeadlines, ngram_range = (1,1), target_column='Target')

Best score: -0.0014909721384406235
Best parameters: min_df=0.060000000000000005, tfidf_min=0.9999999999999999


In [25]:
FakeNews = pd.read_csv('E:/Python/Untitled Folder/fake_oil_news_dataset.csv')
FakeNews = pd.concat([FakeNews, Target['Target']], axis=1)
grid_search(FakeNews, ngram_range = (1,2), target_column='Target')

Best score: -0.001584741669566094
Best parameters: min_df=0.01, tfidf_min=0.5


In [96]:
grid_search(df, n=3, target_column='Target')

Best score: -0.0010618200296039272
Best parameters: min_df=0.17, tfidf_min=0.6


In [97]:
grid_search(df, n=4, target_column='Target')

Best score: -0.0013681127563134996
Best parameters: min_df=0.09999999999999999, tfidf_min=0.5
